In [1]:
import os
import sys

sys.path.insert(0, "../../")

import tensorflow as tf

os.environ["CUDA_VISIBLE_DEVICES"] = "3"

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)])
    except RuntimeError as e:
        print(e)

import pandas as pd
import warnings


warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
pd.set_option('display.max_columns', None)
import pathlib

from lips.benchmark.powergridBenchmark import PowerGridBenchmark
from lips.augmented_simulators.tensorflow_models import LeapNet
from lips.dataset.scaler.powergrid_scaler import PowerGridScaler

from lips.config import ConfigManager
from lips.utils import get_path

# 1. l2rpn_neurips_2020_track1_small grid

In [2]:
# indicate required paths
LIPS_PATH = pathlib.Path().resolve().parent.parent # it is supposed that the notebook had run from getting_started folder
DATA_PATH = LIPS_PATH / "reference_data" / "powergrid" / "l2rpn_neurips_2020_track1_small"
BENCH_CONFIG_PATH = LIPS_PATH / "configurations" / "powergrid" / "benchmarks" / "l2rpn_neurips_2020_track1_small.ini"
SIM_CONFIG_PATH = LIPS_PATH / "configurations" / "powergrid" / "simulators"
BASELINES_PATH = LIPS_PATH / "trained_baselines" / "powergrid"
TRAINED_BASELINES_PATH = LIPS_PATH / "trained_baselines" / "powergrid"
EVALUATION_PATH = LIPS_PATH / "evaluation_results" / "PowerGrid"
LOG_PATH = LIPS_PATH / "lips_logs.log"

## 1.1 Benchmark2: Risk assessment through contingency screening

In [3]:
benchmark2 = PowerGridBenchmark(benchmark_name="Benchmark2",
                                benchmark_path=DATA_PATH,
                                load_data_set=True,
                                log_path=LOG_PATH,
                                config_path=BENCH_CONFIG_PATH
                               )

In [4]:
bench_config = ConfigManager(section_name="Benchmark2", path=BENCH_CONFIG_PATH)
topo_actions = bench_config.get_option("dataset_create_params")["train"]["topo_actions"]

kwargs_tau = []
for el in topo_actions:
     kwargs_tau.append(el["set_bus"]["substations_id"][0])

kwargs_tau

[(26, (2, 1, 1, 2, 1, 1, 1, 1, 1)),
 (26, (2, 2, 2, 1, 2, 1, 1, 1, 1)),
 (26, (2, 2, 1, 1, 2, 2, 1, 1, 1)),
 (23, (2, 2, 2, 2, 1, 1, 2, 2, 1, 2)),
 (23, (2, 2, 1, 2, 1, 1, 2, 2, 1, 1)),
 (23, (2, 1, 1, 1, 2, 2, 2, 2, 2, 2)),
 (23, (2, 2, 1, 1, 2, 2, 2, 1, 2, 2)),
 (16, (2, 1, 1, 1, 2, 1, 1, 1, 2, 2, 2, 1, 2, 1, 1, 1, 2)),
 (16, (2, 1, 1, 2, 2, 2, 2, 1, 1, 2, 2, 1, 1, 2, 2, 1, 1)),
 (16, (2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 1, 1, 2, 2, 1, 2, 1)),
 (16, (2, 2, 2, 2, 2, 1, 1, 2, 2, 1, 1, 2, 2, 2, 1, 2, 2)),
 (16, (2, 1, 1, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 1)),
 (16, (2, 1, 1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2)),
 (16, (2, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2, 1, 1, 2, 1, 1, 2)),
 (16, (1, 2, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)),
 (16, (1, 1, 1, 1, 1, 2, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1)),
 (9, (2, 2, 1, 1, 1, 1, 2)),
 (1, (2, 2, 1, 1, 2, 2)),
 (1, (2, 2, 1, 1, 1, 2)),
 (32, (2, 2, 1, 1, 1)),
 (4, (2, 1, 2, 2, 2, 1)),
 (4, (2, 2, 2, 2, 1, 1)),
 (28, (2, 1, 2, 1, 1))]

## LeapNet

In [5]:
leap_net2 = LeapNet(name="tf_leapnet",
                   bench_config_path=BENCH_CONFIG_PATH,
                   bench_config_name="Benchmark2",
                   sim_config_path=SIM_CONFIG_PATH / "tf_leapnet.ini",
                   sim_config_name="DEFAULT",
                    
                   sizes_enc=(),
                   sizes_main=(400, 400),
                   sizes_out=(),
                   topo_vect_to_tau="given_list",
                   kwargs_tau = kwargs_tau,
                   concatenate_tau = True,
                    
                   loss = {"name": "mse"},
                   lr = 1e-4,
                   layer = "resnet",
                   activation = tf.keras.layers.LeakyReLU(alpha=0.01),
                    
                   scale_main_layer = 400,
                   scale_input_dec_layer = 400,
                   
                   log_path=LOG_PATH,
                   
                   mult_by_zero_lines_pred = False,
                    
                   scaler = PowerGridScaler
                   )

/home/ubuntu/SYSTEMX/seif/seif_dev/LIPS/getting_started/training baselines /../../leap_net/proxy/proxyLeapNet.py:147: UserWarning: We strongly recommend you to get the "line_status" as an input vector
  warnings.warn("We strongly recommend you to get the \"line_status\" as an input vector")


In [6]:
import time
start_time = time.time()

In [ ]:
leap_net2.train(train_dataset=benchmark2.train_dataset,
                val_dataset=benchmark2.val_dataset,
                batch_size = 128,
                epochs = 800)
#epochs=800

2022-07-22 14:10:42.490420: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-22 14:10:42.893130: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5120 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:c3:00.0, compute capability: 8.6


Epoch 1/800
  19/1172 [..............................] - ETA: 10s - loss: 17.5572 - a_or_hat_loss: 2.9653 - a_ex_hat_loss: 2.8893 - p_or_hat_loss: 5.2140 - p_ex_hat_loss: 5.4121 - v_or_hat_loss: 0.5403 - v_ex_hat_loss: 0.5362 - a_or_hat_mae: 1.3429 - a_ex_hat_mae: 1.3191 - p_or_hat_mae: 1.1392 - p_ex_hat_mae: 1.1761 - v_or_hat_mae: 0.6185 - v_ex_hat_mae: 0.6109

2022-07-22 14:10:58.937386: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1172/1172 [==============================] - 14s 11ms/step - loss: 1.6849 - a_or_hat_loss: 0.2574 - a_ex_hat_loss: 0.2494 - p_or_hat_loss: 0.5539 - p_ex_hat_loss: 0.5650 - v_or_hat_loss: 0.0293 - v_ex_hat_loss: 0.0299 - a_or_hat_mae: 0.2791 - a_ex_hat_mae: 0.2730 - p_or_hat_mae: 0.2675 - p_ex_hat_mae: 0.2692 - v_or_hat_mae: 0.0958 - v_ex_hat_mae: 0.0973 - val_loss: 0.8008 - val_a_or_hat_loss: 0.1245 - val_a_ex_hat_loss: 0.1198 - val_p_or_hat_loss: 0.2643 - val_p_ex_hat_loss: 0.2765 - val_v_or_hat_loss: 0.0078 - val_v_ex_hat_loss: 0.0079 - val_a_or_hat_mae: 0.2023 - val_a_ex_hat_mae: 0.1983 - val_p_or_hat_mae: 0.1964 - val_p_ex_hat_mae: 0.1972 - val_v_or_hat_mae: 0.0518 - val_v_ex_hat_mae: 0.0527
Epoch 2/800
1172/1172 [==============================] - 12s 10ms/step - loss: 0.5297 - a_or_hat_loss: 0.0845 - a_ex_hat_loss: 0.0805 - p_or_hat_loss: 0.1740 - p_ex_hat_loss: 0.1784 - v_or_hat_loss: 0.0061 - v_ex_hat_loss: 0.0061 - a_or_hat_mae: 0.1689 - a_ex_hat_mae: 0.1653 - p_or_hat_mae: 0.1

1172/1172 [==============================] - 12s 11ms/step - loss: 0.1018 - a_or_hat_loss: 0.0177 - a_ex_hat_loss: 0.0170 - p_or_hat_loss: 0.0330 - p_ex_hat_loss: 0.0324 - v_or_hat_loss: 8.6938e-04 - v_ex_hat_loss: 8.7320e-04 - a_or_hat_mae: 0.0772 - a_ex_hat_mae: 0.0755 - p_or_hat_mae: 0.0735 - p_ex_hat_mae: 0.0723 - v_or_hat_mae: 0.0167 - v_ex_hat_mae: 0.0166 - val_loss: 0.1280 - val_a_or_hat_loss: 0.0208 - val_a_ex_hat_loss: 0.0201 - val_p_or_hat_loss: 0.0428 - val_p_ex_hat_loss: 0.0425 - val_v_or_hat_loss: 8.9573e-04 - val_v_ex_hat_loss: 8.8485e-04 - val_a_or_hat_mae: 0.0790 - val_a_ex_hat_mae: 0.0784 - val_p_or_hat_mae: 0.0772 - val_p_ex_hat_mae: 0.0768 - val_v_or_hat_mae: 0.0181 - val_v_ex_hat_mae: 0.0180
Epoch 24/800
1172/1172 [==============================] - 12s 10ms/step - loss: 0.0991 - a_or_hat_loss: 0.0173 - a_ex_hat_loss: 0.0166 - p_or_hat_loss: 0.0321 - p_ex_hat_loss: 0.0314 - v_or_hat_loss: 8.3772e-04 - v_ex_hat_loss: 8.1720e-04 - a_or_hat_mae: 0.0762 - a_ex_hat_mae: 0

...

1172/1172 [==============================] - 13s 11ms/step - loss: 0.0139 - a_or_hat_loss: 0.0039 - a_ex_hat_loss: 0.0037 - p_or_hat_loss: 0.0031 - p_ex_hat_loss: 0.0030 - v_or_hat_loss: 5.8977e-05 - v_ex_hat_loss: 7.1913e-05 - a_or_hat_mae: 0.0361 - a_ex_hat_mae: 0.0350 - p_or_hat_mae: 0.0252 - p_ex_hat_mae: 0.0249 - v_or_hat_mae: 0.0037 - v_ex_hat_mae: 0.0038 - val_loss: 0.0539 - val_a_or_hat_loss: 0.0097 - val_a_ex_hat_loss: 0.0091 - val_p_or_hat_loss: 0.0175 - val_p_ex_hat_loss: 0.0174 - val_v_or_hat_loss: 7.9814e-05 - val_v_ex_hat_loss: 9.1198e-05 - val_a_or_hat_mae: 0.0448 - val_a_ex_hat_mae: 0.0426 - val_p_or_hat_mae: 0.0334 - val_p_ex_hat_mae: 0.0326 - val_v_or_hat_mae: 0.0036 - val_v_ex_hat_mae: 0.0038
Epoch 640/800
1172/1172 [==============================] - 13s 11ms/step - loss: 0.0146 - a_or_hat_loss: 0.0040 - a_ex_hat_loss: 0.0038 - p_or_hat_loss: 0.0034 - p_ex_hat_loss: 0.0033 - v_or_hat_loss: 6.1481e-05 - v_ex_hat_loss: 7.4588e-05 - a_or_hat_mae: 0.0368 - a_ex_hat_mae: 

1172/1172 [==============================] - 13s 11ms/step - loss: 0.0136 - a_or_hat_loss: 0.0039 - a_ex_hat_loss: 0.0036 - p_or_hat_loss: 0.0030 - p_ex_hat_loss: 0.0030 - v_or_hat_loss: 5.5644e-05 - v_ex_hat_loss: 6.8020e-05 - a_or_hat_mae: 0.0360 - a_ex_hat_mae: 0.0350 - p_or_hat_mae: 0.0250 - p_ex_hat_mae: 0.0247 - v_or_hat_mae: 0.0037 - v_ex_hat_mae: 0.0038 - val_loss: 0.0546 - val_a_or_hat_loss: 0.0096 - val_a_ex_hat_loss: 0.0091 - val_p_or_hat_loss: 0.0179 - val_p_ex_hat_loss: 0.0178 - val_v_or_hat_loss: 7.8542e-05 - val_v_ex_hat_loss: 9.1549e-05 - val_a_or_hat_mae: 0.0447 - val_a_ex_hat_mae: 0.0440 - val_p_or_hat_mae: 0.0360 - val_p_ex_hat_mae: 0.0360 - val_v_or_hat_mae: 0.0039 - val_v_ex_hat_mae: 0.0041
Epoch 662/800
1172/1172 [==============================] - 13s 11ms/step - loss: 0.0163 - a_or_hat_loss: 0.0043 - a_ex_hat_loss: 0.0041 - p_or_hat_loss: 0.0039 - p_ex_hat_loss: 0.0039 - v_or_hat_loss: 6.5888e-05 - v_ex_hat_loss: 7.8856e-05 - a_or_hat_mae: 0.0381 - a_ex_hat_mae: 

1172/1172 [==============================] - 13s 11ms/step - loss: 0.0136 - a_or_hat_loss: 0.0038 - a_ex_hat_loss: 0.0036 - p_or_hat_loss: 0.0030 - p_ex_hat_loss: 0.0030 - v_or_hat_loss: 5.5111e-05 - v_ex_hat_loss: 6.7481e-05 - a_or_hat_mae: 0.0358 - a_ex_hat_mae: 0.0349 - p_or_hat_mae: 0.0250 - p_ex_hat_mae: 0.0247 - v_or_hat_mae: 0.0036 - v_ex_hat_mae: 0.0038 - val_loss: 0.0525 - val_a_or_hat_loss: 0.0095 - val_a_ex_hat_loss: 0.0090 - val_p_or_hat_loss: 0.0170 - val_p_ex_hat_loss: 0.0168 - val_v_or_hat_loss: 6.8193e-05 - val_v_ex_hat_loss: 8.0597e-05 - val_a_or_hat_mae: 0.0450 - val_a_ex_hat_mae: 0.0429 - val_p_or_hat_mae: 0.0324 - val_p_ex_hat_mae: 0.0318 - val_v_or_hat_mae: 0.0035 - val_v_ex_hat_mae: 0.0037
Epoch 684/800
1172/1172 [==============================] - 13s 11ms/step - loss: 0.0135 - a_or_hat_loss: 0.0038 - a_ex_hat_loss: 0.0036 - p_or_hat_loss: 0.0030 - p_ex_hat_loss: 0.0030 - v_or_hat_loss: 5.2907e-05 - v_ex_hat_loss: 6.4710e-05 - a_or_hat_mae: 0.0361 - a_ex_hat_mae: 

1172/1172 [==============================] - 13s 11ms/step - loss: 0.0123 - a_or_hat_loss: 0.0036 - a_ex_hat_loss: 0.0034 - p_or_hat_loss: 0.0026 - p_ex_hat_loss: 0.0026 - v_or_hat_loss: 4.8582e-05 - v_ex_hat_loss: 5.9870e-05 - a_or_hat_mae: 0.0347 - a_ex_hat_mae: 0.0337 - p_or_hat_mae: 0.0235 - p_ex_hat_mae: 0.0233 - v_or_hat_mae: 0.0034 - v_ex_hat_mae: 0.0036 - val_loss: 0.0577 - val_a_or_hat_loss: 0.0102 - val_a_ex_hat_loss: 0.0096 - val_p_or_hat_loss: 0.0191 - val_p_ex_hat_loss: 0.0186 - val_v_or_hat_loss: 8.9195e-05 - val_v_ex_hat_loss: 9.7492e-05 - val_a_or_hat_mae: 0.0476 - val_a_ex_hat_mae: 0.0459 - val_p_or_hat_mae: 0.0384 - val_p_ex_hat_mae: 0.0368 - val_v_or_hat_mae: 0.0042 - val_v_ex_hat_mae: 0.0044
Epoch 706/800
1172/1172 [==============================] - 13s 11ms/step - loss: 0.0139 - a_or_hat_loss: 0.0038 - a_ex_hat_loss: 0.0036 - p_or_hat_loss: 0.0032 - p_ex_hat_loss: 0.0031 - v_or_hat_loss: 5.5244e-05 - v_ex_hat_loss: 6.6531e-05 - a_or_hat_mae: 0.0360 - a_ex_hat_mae: 

1172/1172 [==============================] - 13s 11ms/step - loss: 0.0137 - a_or_hat_loss: 0.0038 - a_ex_hat_loss: 0.0036 - p_or_hat_loss: 0.0031 - p_ex_hat_loss: 0.0030 - v_or_hat_loss: 5.3458e-05 - v_ex_hat_loss: 6.5451e-05 - a_or_hat_mae: 0.0361 - a_ex_hat_mae: 0.0351 - p_or_hat_mae: 0.0253 - p_ex_hat_mae: 0.0250 - v_or_hat_mae: 0.0036 - v_ex_hat_mae: 0.0038 - val_loss: 0.0546 - val_a_or_hat_loss: 0.0097 - val_a_ex_hat_loss: 0.0092 - val_p_or_hat_loss: 0.0179 - val_p_ex_hat_loss: 0.0177 - val_v_or_hat_loss: 7.0750e-05 - val_v_ex_hat_loss: 8.4669e-05 - val_a_or_hat_mae: 0.0450 - val_a_ex_hat_mae: 0.0438 - val_p_or_hat_mae: 0.0342 - val_p_ex_hat_mae: 0.0344 - val_v_or_hat_mae: 0.0036 - val_v_ex_hat_mae: 0.0038
Epoch 728/800
1172/1172 [==============================] - 13s 11ms/step - loss: 0.0131 - a_or_hat_loss: 0.0037 - a_ex_hat_loss: 0.0035 - p_or_hat_loss: 0.0029 - p_ex_hat_loss: 0.0028 - v_or_hat_loss: 5.3212e-05 - v_ex_hat_loss: 6.5276e-05 - a_or_hat_mae: 0.0356 - a_ex_hat_mae: 

In [ ]:
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
leap_net2.summary()

In [ ]:
tf.keras.utils.plot_model(
    leap_net2._model,
    show_shapes=True,
    show_dtype=True,
    show_layer_activations= True)

In [ ]:
leap_net2.visualize_convergence()

In [ ]:
tf_leapnet_metrics2 = benchmark2.evaluate_simulator(augmented_simulator=leap_net2,
                                              eval_batch_size=128,
                                              dataset="all",
                                              shuffle=False,
                                              save_predictions=True
                                             )

In [ ]:
SAVE_PATH = get_path(TRAINED_BASELINES_PATH, benchmark1)
leap_net2.save(SAVE_PATH)

In [ ]:
import numpy as np
eval_metrics = tf_leapnet_metrics2

## A 

In [ ]:
a_test_list = [eval_metrics["test"]["ML"]["mape_90_avg"]["a_or"], eval_metrics["test"]["ML"]["mape_90_avg"]["a_ex"]]
a_ood_list = [eval_metrics["test_ood_topo"]["ML"]["mape_90_avg"]["a_or"], eval_metrics["test_ood_topo"]["ML"]["mape_90_avg"]["a_ex"]]

In [ ]:
print("ML : MAPE90 --> A : mean :{}, std :{}".format(np.mean(a_test_list), np.std(a_test_list)))
print("OOD : MAPE90 --> A : mean :{}, std :{}".format(np.mean(a_ood_list), np.std(a_ood_list)))

## P

In [ ]:
p_test_list = [eval_metrics["test"]["ML"]["mape_avg"]["p_or"], eval_metrics["test"]["ML"]["mape_avg"]["p_ex"]]
p_ood_list = [eval_metrics["test_ood_topo"]["ML"]["mape_avg"]["p_or"], eval_metrics["test_ood_topo"]["ML"]["mape_avg"]["p_ex"]]

In [ ]:
print("ML : MAPE --> P : mean :{}, std :{}".format(np.mean(p_test_list), np.std(p_test_list)))
print("OOD : MAPE --> P : mean :{}, std :{}".format(np.mean(p_ood_list), np.std(p_ood_list)))

## V 

In [ ]:
v_test_list = [eval_metrics["test"]["ML"]["MAE_avg"]["v_or"], eval_metrics["test"]["ML"]["MAE_avg"]["v_ex"]]
v_ood_list = [eval_metrics["test_ood_topo"]["ML"]["MAE_avg"]["v_or"], eval_metrics["test_ood_topo"]["ML"]["MAE_avg"]["v_ex"]]

In [ ]:
print("ML : MAE --> V : mean :{}, std :{}".format(np.mean(v_test_list), np.std(v_test_list)))
print("OOD : MAE --> V : mean :{}, std :{}".format(np.mean(v_ood_list), np.std(v_ood_list)))

## Inf speed-up

In [ ]:
print("ML : Inf speed-up :", eval_metrics["test"]["ML"]["TIME_INF"])

## Physics ML

In [ ]:
print("1) Current positivity violation:", (eval_metrics["test"]["Physics"]["CURRENT_POS"]["a_or"]["Violation_proportion"]+eval_metrics["test"]["Physics"]["CURRENT_POS"]["a_ex"]["Violation_proportion"])/2)#["a_or"]["Violation_proportion"]
print("2) Voltage positivity violation:", (eval_metrics["test"]["Physics"]["VOLTAGE_POS"]["v_or"]["Violation_proportion"]+eval_metrics["test"]["Physics"]["VOLTAGE_POS"]["v_ex"]["Violation_proportion"])/2)
print("3) Loss positivity violation:", eval_metrics["test"]["Physics"]["LOSS_POS"]["violation_proportion"])
print("4) Disconnected lines violation:", eval_metrics["test"]["Physics"]["DISC_LINES"])
print("5) Violation of loss to be between [1,4]% of production:", eval_metrics["test"]["Physics"]["CHECK_LOSS"]["violation_percentage"])
print("6) Violation of global conservation: {}% and its weighted mape: {}".format(eval_metrics["test"]["Physics"]["CHECK_GC"]["violation_percentage"], eval_metrics["test"]["Physics"]["CHECK_GC"]["wmape"]))
print("7) Violation of local conservation: {}% and its weighted mape: {}".format(eval_metrics["test"]["Physics"]["CHECK_LC"]["violation_percentage"], eval_metrics["test"]["Physics"]["CHECK_LC"]["mape"]))
print("8) Violation proportion of voltage equality at subs:", eval_metrics["test"]["Physics"]["CHECK_VOLTAGE_EQ"]["prop_voltages_violation"])

## Physics OOD

In [ ]:
print("1) Current positivity violation:", (eval_metrics["test_ood_topo"]["Physics"]["CURRENT_POS"]["a_or"]["Violation_proportion"]+eval_metrics["test_ood_topo"]["Physics"]["CURRENT_POS"]["a_ex"]["Violation_proportion"])/2)#["a_or"]["Violation_proportion"]
print("2) Voltage positivity violation:", (eval_metrics["test_ood_topo"]["Physics"]["VOLTAGE_POS"]["v_or"]["Violation_proportion"]+eval_metrics["test_ood_topo"]["Physics"]["VOLTAGE_POS"]["v_ex"]["Violation_proportion"])/2)
print("3) Loss positivity violation:", eval_metrics["test_ood_topo"]["Physics"]["LOSS_POS"]["violation_proportion"])
print("4) Disconnected lines violation:", eval_metrics["test_ood_topo"]["Physics"]["DISC_LINES"])
print("5) Violation of loss to be between [1,4]% of production:", eval_metrics["test_ood_topo"]["Physics"]["CHECK_LOSS"]["violation_percentage"])
print("6) Violation of global conservation: {}% and its weighted mape: {}".format(eval_metrics["test_ood_topo"]["Physics"]["CHECK_GC"]["violation_percentage"], eval_metrics["test_ood_topo"]["Physics"]["CHECK_GC"]["wmape"]))
print("7) Violation of local conservation: {}% and its weighted mape: {}".format(eval_metrics["test_ood_topo"]["Physics"]["CHECK_LC"]["violation_percentage"], eval_metrics["test_ood_topo"]["Physics"]["CHECK_LC"]["mape"]))
print("8) Violation proportion of voltage equality at subs:", eval_metrics["test_ood_topo"]["Physics"]["CHECK_VOLTAGE_EQ"]["prop_voltages_violation"])